In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
torch.manual_seed(1)
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
def load_data(train_batch_size, test_batch_size):
    # Fetch training data: total 60000 samples
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train = True, download=True,
                       transform=transforms.Compose([
                           transforms.Resize((32, 32)),
                           transforms.ToTensor()
                       ])),
        batch_size = train_batch_size, shuffle=True)

    # Fetch test data: total 10000 samples
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train = False, transform=transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor()
        ])),
        batch_size = test_batch_size, shuffle=True)

    return (train_loader, test_loader)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # flatten as one dimension
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        x = self.fc3(x)
        return x

In [5]:
def sensitivity(model):
    fNormAll = 0
    counter = 0
    for p in model.parameters():
        grad = 0.0
        if p.grad is not None:
            grad = p.grad
            fNorm = torch.linalg.norm(grad).numpy()
            fNormAll += fNorm
            counter += 1
    return fNormAll / counter

In [6]:
def define_optimizer(model):
    return optim.Adam(model.parameters(), lr = 0.001)

In [7]:
def train(model, optimizer, train_loader):
    model.train()
    loss_fn = torch.nn.CrossEntropyLoss()
    correct = 0
    train_loss = 0
    counter = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        counter+=1
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()

        # Forward propagation
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        
        optimizer.step()
        
        sens = sensitivity(model)
        train_loss += loss.data

        pred = np.argmax(output.data, axis=1)
        correct += np.equal(pred, target.data).sum()

    train_loss /= counter
    acc = 100.0 * correct / len(train_loader.dataset)
    return train_loss, acc, sens

In [8]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    loss_fn = torch.nn.CrossEntropyLoss(size_average=False)

    for data, target in test_loader:
        data, target = Variable(data), Variable(target)
        
        output = model(data)
        test_loss += loss_fn(output, target).data

        pred = np.argmax(output.data, axis=1)
        correct += np.equal(pred, target.data).sum()

    test_loss /= len(test_loader.dataset)
    acc = 100.0 * correct / len(test_loader.dataset)
    return test_loss, acc

In [9]:
def compute(model, optimizer):
    for epoch in range(1, epochs + 1):
        tr_loss, train_acc, sens = train(model, optimizer, train_loader)
        t_loss, test_acc = test(model, test_loader)
    print(sens)
    #sensitivity(model)
    sensList.append(sens)
    loss_train_arr.append(tr_loss)
    loss_test_arr.append(t_loss)
    print("Model Train loss: ", tr_loss)
    print("Model Test loss: ", t_loss)
    train_acc_arr.append(train_acc)
    test_acc_arr.append(test_acc)

In [10]:
loss_train_arr = []
loss_test_arr = []
test_acc_arr = []
train_acc_arr = []
sensList= []
epochs = 50

In [11]:
batch_size = [5, 25, 125, 500, 1500]
for batch in batch_size:
    torch.manual_seed(1)
    train_loader, test_loader = load_data(batch, batch)
    model1 = CNN()
    optimizer = define_optimizer(model1)
    compute(model1, optimizer)

C:\Users\nikhi\anaconda3\envs\deeplearning\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
